# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [4]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [5]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [133]:
#def P1(num_examples=5):
### STUDENT START ###
for i in range (0,5):
    print "-"*20
    print train_labels[i], newsgroups_train.target_names[train_labels[i]]
    print "-"*20
    print train_data[i]
    print

### STUDENT END ###
#P1()

--------------------
1 comp.graphics
--------------------
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

--------------------
3 talk.religion.misc
--------------------


Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [120]:
#def P2():
### STUDENT START ###

print "a)"
# What is the size of the vocabulary?
cv = CountVectorizer(input='content')
X = cv.fit_transform(train_data)
vocab_size = X.shape[1]
print "Vocabulary size: {}".format( vocab_size )

# What is the average number of non-zero features per example?
total_nonzero = 0
num_examples = 0
for x in X:
    total_nonzero += x.nnz
    num_examples += 1
print "Average number of non-zero features per example: {:0.2f}".format(
        float(total_nonzero) / num_examples )

# What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.
print "Fraction of non-zero entries in the matrix: {:0.4f}%".format(
        100 * float(total_nonzero) / (X.shape[0] * X.shape[1]) )

print "\nb)"
# What are the 0th and last feature strings (in alphabetical order)?
# Hint: use the vectorizer's get_feature_names function.
print '0th feature name: "{}"'.format(cv.get_feature_names()[0])
print 'Last feature name: "{}"'.format(cv.get_feature_names()[vocab_size-1])

print "\nc)"
# Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"].
# Confirm the training vectors are appropriately shaped.
vocab = ["atheism", "graphics", "space", "religion"]
cv = CountVectorizer(input='content', vocabulary=vocab)
X = cv2.fit_transform(train_data)
vocab_size = X.shape[1]
print "Vocabulary size: {}".format( vocab_size )

# Now what's the average number of non-zero features per example?
total_nonzero = 0
num_examples = 0
for x in X:
    total_nonzero += x.nnz
    num_examples += 1
print "Average number of non-zero features per example: {:0.2f}".format( float(total_nonzero) / num_examples )

print "\nd)"
# Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram
# and trigram character features. What size vocabulary does this yield?
cv = CountVectorizer(input='content', analyzer='char', ngram_range=(2,3))
X = cv.fit_transform(train_data)
vocab_size = X.shape[1]
print "Bigram/trigram vocabulary size: {}".format( vocab_size )

print "\ne)"
# Use the "min_df" argument to prune words that appear in fewer than 10 documents.
# What size vocabulary does this yield?
cv = CountVectorizer(input='content', min_df=10)
X = cv.fit_transform(train_data)
vocab_size = X.shape[1]
print "Pruned vocabulary size: {}".format( vocab_size )

print "\nf)"
# Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary?
# Hint: build a vocabulary for both train and dev and look at the size of the difference.
cv = CountVectorizer(input='content')
cv.fit(train_data)
train_set = set(cv.get_feature_names())

cv = CountVectorizer(input='content')
cv.fit(dev_data)
dev_set = set(cv.get_feature_names())

# dev_vocab_size = cv.transform(dev_data).shape[1]
diff_size = len(train_set.difference(dev_set))

print "Fraction of words from dev_data missing from vocab:"
print "    {} / {} =  {:.2f}%".format( diff_size, dev_vocab_size, 100 * float(diff_size) / dev_vocab_size)

### STUDENT END ###
#P2()

a)
Vocabulary size: 26879
Average number of non-zero features per example: 96.71
Fraction of non-zero entries in the matrix: 0.3598%

b)
0th feature name: "00"
Last feature name: "zyxel"

c)
Vocabulary size: 4
Average number of non-zero features per example: 0.27

d)
Bigram/trigram vocabulary size: 35478

e)
Pruned vocabulary size: 3064

f)
Fraction of words from dev_data missing from vocab:
    14660 / 16246 =  90.24%


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [195]:
#def P3():
### STUDENT START ###

cv = CountVectorizer(input='content')
cv.fit(train_data)

X = cv.transform(train_data)
Y = cv.transform(dev_data)

######## K Nearest Neighbors ########
knn = KNeighborsClassifier()

# k values to test
ks = {'n_neighbors': [1, 3, 5, 7, 9]}

# perform a grid search on training data
gs = GridSearchCV(estimator=knn, param_grid=ks)
gs.fit(X, train_labels)
f1 = metrics.f1_score(dev_labels, gs.predict(Y), average='weighted')

# print best results
print("K Nearest Neighbors: best k = {}, accuracy={:.4f}, f1={:.4f}\n".format(
        gs.best_estimator_.n_neighbors, gs.best_score_, f1))


######## Naive Bayes ########
bnb = BernoulliNB()

# alpha values to test
alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}

# perform a grid search on training data
gs = GridSearchCV(estimator=bnb, param_grid=alphas)
gs.fit(X, train_labels)
f1 = metrics.f1_score(dev_labels, gs.predict(Y), average='weighted')

# print best results
print("Naive Bayes: best alpha = {}, accuracy={:.4f}, f1={:.4f}\n".format(
        gs.best_estimator_.alpha, gs.best_score_, f1))


######## Logistic Regression ########
lr = LogisticRegression(penalty='l2')

# C values to test
cs = {'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}

# perform a grid search on training data
gs = GridSearchCV(estimator=lr, param_grid=cs)
gs.fit(X, train_labels)
f1 = metrics.f1_score(dev_labels, gs.predict(Y), average='weighted')

# print best results
print("Logistic Regression: best C = {}, accuracy={:.4f}, f1={:.4f}\n".format(
        gs.best_estimator_.C, gs.best_score_, f1))


# Output the sum of the squared weight values for each class for each setting of the C parameter. 

lr = LogisticRegression(penalty='l2')

# C values to test
cs = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]


for c in cs:
    lr.set_params(C=c)
    lr.fit(X, train_labels)
    squared_weights = np.zeros(len(categories))
    for category in range(0, len(categories)):
        for weight in lr.coef_[category]:
            squared_weights[category] += weight**2
        print "C={}, Class={} : Σ weight² = {:.6f}".format(c, category, squared_weights[category])



### STUDENT END ###
#P3()

K Nearest Neighbors: best k = 9, accuracy=0.4105, f1=0.4366

Naive Bayes: best alpha = 0.0001, accuracy=0.7866, f1=0.6856

Logistic Regression: best C = 0.5, accuracy=0.7714, f1=0.7085

C=0.0001, Class=0 : Σ weight² = 0.007702
C=0.0001, Class=1 : Σ weight² = 0.011941
C=0.0001, Class=2 : Σ weight² = 0.009435
C=0.0001, Class=3 : Σ weight² = 0.009103
C=0.001, Class=0 : Σ weight² = 0.165093
C=0.001, Class=1 : Σ weight² = 0.200953
C=0.001, Class=2 : Σ weight² = 0.180671
C=0.001, Class=3 : Σ weight² = 0.187243
C=0.01, Class=0 : Σ weight² = 2.541488
C=0.01, Class=1 : Σ weight² = 2.939709
C=0.01, Class=2 : Σ weight² = 2.862469
C=0.01, Class=3 : Σ weight² = 2.250029
C=0.1, Class=0 : Σ weight² = 27.136235
C=0.1, Class=1 : Σ weight² = 24.659567
C=0.1, Class=2 : Σ weight² = 27.458723
C=0.1, Class=3 : Σ weight² = 23.025997
C=0.5, Class=0 : Σ weight² = 102.513464
C=0.5, Class=1 : Σ weight² = 83.102160
C=0.5, Class=2 : Σ weight² = 98.982650
C=0.5, Class=3 : Σ weight² = 88.998614
C=1.0, Class=0 : Σ we

a. Nearest neighbors does not work well because the feature set is a sparse matrix.  

b. My logistic regression has a lower accuracy but higher f1-score than Naive Bayes, so not sure what that means.

c. The sum of squared weights increases as C increases.



(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [101]:
#def P4():
### STUDENT START ###

cvs = []
cvs.append(CountVectorizer(input='content'))
cvs.append(CountVectorizer(input='content', analyzer='char', ngram_range=(2,3)))

for cv in cvs:
    # train model
    X = cv.fit_transform(train_data)
    lr = LogisticRegression(penalty='l2', C=0.5)
    lr.fit(X, train_labels)

    # print header row
    for j in range(0, len(categories)):
        print "{:8}".format(j),
    print
    for j in range(0, len(categories)):
        print "{:8}".format("-"*8),
    print

    # Print 20 rows and 4 columns that shows the weight for each of these features for each of the labels. 
    for category in range(0, len(categories)):
        # get a reverse-sorted list of indices for the weights
        indices = np.argsort(lr.coef_[category])[::-1]

        for i in indices[0:5]:
            for j in range(0, len(categories)):
                print "{:8.4f}".format(lr.coef_[j][i]),
            print
    
    print


### STUDENT END ###
#P4()

       0        1        2        3
-------- -------- -------- --------
  0.8341  -0.0934  -0.2674  -0.6375
  0.8236  -0.1913  -0.2865  -0.3872
  0.7984  -0.5083  -0.6588  -0.0505
  0.7947  -0.3506  -0.3672  -0.3768
  0.7238  -0.2493  -0.2725  -0.2704
 -0.6409   1.6260  -1.1030  -0.6264
 -0.4695   1.1169  -0.6722  -0.3805
 -0.2753   1.0596  -0.6827  -0.5063
 -0.3035   0.9418  -0.5711  -0.3175
  0.0714   0.8444  -0.5695  -0.3963
 -1.0615  -1.1163   1.8641  -0.9721
 -0.3503  -0.5596   1.0127  -0.4990
 -0.4654  -0.4105   0.8517  -0.4016
 -0.3776  -0.3968   0.7839  -0.2807
 -0.2975  -0.3220   0.7373  -0.2918
 -0.6074  -0.3153  -0.4044   0.9296
 -0.4830  -0.3390  -0.2475   0.9256
 -0.4288  -0.0969  -0.2307   0.8469
 -0.2440  -0.2175  -0.3750   0.7547
 -0.6495  -0.0657  -0.1242   0.7429

       0        1        2        3
-------- -------- -------- --------
  0.3613  -0.0927  -0.0010  -0.0901
  0.3538  -0.0492  -0.2774  -0.0923
  0.3363  -0.1946  -0.0384  -0.0139
  0.3344   0.0002  -0.0439 

Surprising feature: the weights are lower with bigram features.

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [183]:
#def empty_preprocessor(s):
#    return s

#def better_preprocessor(s):
### STUDENT START ###

def better_preprocessor(s):
    # lowercase everything
    s = s.lower()

    # replace numbers with a single '9' token
    s = re.sub(r'[0-9]+', '9', s)
    
    # de-pluralize (remove trailing 's' characters)
    s = re.sub(r's\b', '', s)
    
    # remove trailing 'ing' from words
    s = re.sub(r'ing\b', '', s)
    
    # remove anything not alphanumeric
    s = re.sub(r'[^a-zA-Z0-9 ]', ' ', s)

    # shorten words longer than 6 characters
    s = re.sub(r'([\w]{8})[\w]+', r'\1', s)

    return s

### STUDENT END ###


#def P5():
### STUDENT START ###


cvs = []
cvs.append(CountVectorizer(input='content'))
cvs.append(CountVectorizer(input='content', preprocessor=better_preprocessor))

for cv in cvs:

    # learn a vocabulary for the training data
    cv.fit(train_data)
    vocab_size = len(cv.vocabulary_)

    # fit the training data
    bnb = BernoulliNB(alpha=0.00001)
    bnb.fit(cv.transform(train_data), train_labels)
    
    # test the model on dev_data
    f1 = metrics.f1_score(dev_labels, bnb.predict( cv.transform(dev_data) ), average='weighted')
    print "Vocabulary size = {}, f1-metric = {:.4f}".format( vocab_size, f1)


### STUDENT END ###
#P5()

Vocabulary size = 26879, f1-metric = 0.6878
Vocabulary size = 19433, f1-metric = 0.6903


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [6]:
#def P6():
# Keep this random seed here to make comparison easier.
#np.random.seed(0)

### STUDENT START ###

# plt.scatter(vocabulary_size,scores)
# plt.xlim(min(vocabulary_size), max(vocabulary_size))
# plt.xlabel('Vocabulary size')
# plt.ylabel('Accuracy')
# plt.show()

# For P6, what values of C are people using? I’m using: C = np.arange(1,200,1)
# did you set the tolerance, tol = .01?

## office hours:
## vectorizer.get_feature_names()

cv = CountVectorizer(input='content')
cv.fit(train_data)

X = cv.transform(train_data)
Y = cv.transform(dev_data)

# C values to test
cs = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

lrs = []
lrs.append( LogisticRegression(penalty='l1') )
lrs.append( LogisticRegression(penalty='l2') )

for lr in lrs:
    for c in cs:
        lr.set_params(C=c)
        lr.fit(X, train_labels)
        nonzero_weights = 0
        for category in range(0, len(categories)):
            for weight in lr.coef_[category]:
                if weight > 0:
                    nonzero_weights += 1
        print "C={} : non-zero weights = {}".format(c, nonzero_weights)
    print

### STUDENT END ###
#P6()

C=0.001 : non-zero weights = 0
C=0.01 : non-zero weights = 14
C=0.1 : non-zero weights = 208
C=1.0 : non-zero weights = 883
C=10.0 : non-zero weights = 1626
C=100.0 : non-zero weights = 2328

C=0.001 : non-zero weights = 40551
C=0.01 : non-zero weights = 41281
C=0.1 : non-zero weights = 44060
C=1.0 : non-zero weights = 45184
C=10.0 : non-zero weights = 45386
C=100.0 : non-zero weights = 45893



In [19]:
# reduce the size of the vocabulary by keeping only those features that have
# at least one non-zero weight and retrain a model using "l2".

# TODO: figure out how to trim the vocabulary

for f in range(len(lr.coef_[0])):
    nonzero = False
    for c in range(len(categories)):
        if lr.coef_[c][f] != 0:
            nonzero = True
            break
    if nonzero == True:
        continue
    print "Prune {}".format(f)
    


(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [40]:
#def P7():
### STUDENT START ###

# fit a TFIDF vectorizer and train logistic regression
tv = TfidfVectorizer()
tv.fit(train_data)

lr = LogisticRegression(penalty='l1', C=100)
lr.fit( tv.transform(train_data), train_labels )

# predict values from our model
preds = lr.predict_proba( tv.transform(dev_data) )

# calculate R ratio for our predictions
R = []
for i in range(len(preds)):
    max_prob = preds[i].max()
    prob_correct = preds[i][dev_labels[i]]
    R.append( max_prob / prob_correct )

# get a reverse-sorted list of indices for R
indices = np.argsort(R)[::-1]

# for the top 3 documents, print the category followed by the document
for index in indices[0:3]:
    print "-"*30
    print "True:      " + categories[dev_labels[index]]
    print "Predicted: " + categories[np.argmax(preds[index])]
    print "-"*30
    print dev_data[index]
    print


#f1 = metrics.f1_score(dev_labels, lr.predict( tv.transform(dev_data) ), average='weighted')
#print f1

## STUDENT END ###
#P7()

------------------------------
True:      alt.atheism
Predicted: comp.graphics
------------------------------

The 24 children were, of course, killed by a lone gunman in a second story
window, who fired eight bullets in the space of two seconds...


------------------------------
True:      sci.space
Predicted: talk.religion.misc
------------------------------

I don't think so.  The assumption is there.  If it turns out that
their software has a virus, then it is up to you to prove that fact
to a court to get any damages.  You are theoretically suppossed to 
be able to get damages for that, but you have to give some evidence
that the virus came from that software.  But since the computer
company is the defendent, they are uninvolved until proven guilty.

Please, not Pascal!  NOOOOO!! ;)


------------------------------
True:      sci.space
Predicted: alt.atheism
------------------------------
: >But if entertainment (company) sell computer programs saying they are virus 
: >safe. Doe

It is possible that the word "window" is associated with graphics, leading to a false prediction.  "Belief" and "believers" could be associated with atheism.  More training data could help improve learning.

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.